# Mini-projet : Ruban perforé

Un [ruban perforé](https://fr.wikipedia.org/wiki/Ruban_perfor%C3%A9) est un support ayant permis de stocker de grandes quantités de données aux débuts de l'informatique.

Il se présente sous la forme d'un long ruban de papier percé de trous circulaires.

![Exemple de ruban perforé](https://ntoulzac.github.io/Cours-NSI-Terminale/premiere_NSI/Images/ruban_long.png)

## Partie A - Lecture à la main d'un court message

L'objectif de cette première partie est de comprendre le fonctionnement d'un ruban perforé et de trouver quel message est stocké sur le morceau suivant :

![Autre exemple de ruban perforé](https://ntoulzac.github.io/Cours-NSI-Terminale/premiere_NSI/Images/ruban_court.png)

Sur chaque ruban se trouve un alignement de petits trous qui permettent de faire défiler le ruban. Ces trous, représentés en gris clair sur le dessin, ne codent aucune information et ne sont donc pas à prendre en compte.

Chaque caractère du message est représenté par une colonne, par exemple celle qui est encadrée en rouge sur le dessin. Sur chaque colonne se trouvent sept emplacements qui peuvent être troués ou non et qui représentent les sept bits de l'écriture binaire d'un nombre compris entre 0 et 127.

Les bits sont positionnés de haut en bas par poids croissant : le bit de poids le plus faible est en haut et le bit de poids le plus fort est en bas. Un trou correspond à un `1` et une absence de trou correspond à un `0`.

Dans le cadre rouge, par exemple, on peut lire les sept bits `1000010` puisqu'il n'y a que deux trous (en première et en avant dernière positions).

Il ne reste plus qu'à retrouver quel entier a pour écriture binaire `1000010` : c'est l'entier `66`. Enfin, le caractère associé se lit dans la table ASCII : c'est la lettre `'B'`.

**Question 1 :** Le deuxième caractère du message est encodé par l'entier `1110010` écrit en binaire. Quelle est la valeur décimale de ce nombre et à quel caractère correspond-il dans la table ASCII ?

**Réponse :** `1110010` est l'écriture binaire de l'entier `114` car $64 + 32 + 16 + 2 = 114$. Le caractère correspondant à l'entier `114` dans la table ASCII est `'r'`.

**Question 2 :** Retrouver le message complet stocké sur ce ruban, en expliquant votre démarche.

**Réponse :** Nous avons déjà vu que les deux premiers caractères sont `'Br'`.

Le troisième caractère est associé à l'entier `1000001` écrit en binaire. $64 + 1 = 65$ et l'entier `65` correspond au caractère `'A'` dans la table ASCII.

Le quatrième caractère est associé à l'entier `1110110` écrit en binaire. $64 + 32 + 16 + 4 + 2 = 118$ et l'entier `118` correspond au caractère `'v'` dans la table ASCII.

Le cinquième caractère est associé à l'entier `1101111` écrit en binaire. $127 - 16 = 111$ et l'entier `111` correspond au caractère `'o'` dans la table ASCII.

Le message complet est donc `'BrAvo'`.

## Partie B - Lecture automatisée d'un long message

L'objectif de cette seconde partie est d'automatiser la lecture pour pouvoir déchiffrer des rubans beaucoup plus longs.

On utilisera le module `PIL` qui peut être installé si nécessaire en exécutant la cellule suivante :

In [ ]:
import sys
!{sys.executable} -m pip install Pillow

On donne une première fonction.

In [1]:
from PIL import Image

def est_sombre(pixel):
    """
    Détermine si un pixel est sombre ou pas.
    - Entrée : pixel (p-uplet constitué de trois entiers compris entre 0 et 255, composantes RVB d'un pixel)
    - Sortie : (booléen, True si le pixel est sombre, False sinon)
    """
    return pixel[0] + pixel[1] + pixel[2] < 200

**Question 3 :** Dans les fichiers PNG où sont stockées les images de ruban, les composantes RVB du gris utilisé pour dessiner les trous sont `(64, 64, 64)`. Justifier ce que renvoie la fonction `est_sombre` pour un pixel situé dans un trou.

**Réponse :** Si les trois composantes RVB du pixel sont égales à `64`,  leur somme `pixel[0] + pixel[1] + pixel[2]` vaut `192` et est donc strictement inférieure à `200`. La fonction `est_sombre` renvoie donc `True` dans ce cas.

On donne deux autres fonctions.

In [2]:
def lecture_bits_colonne(image, n):
    """
    Détermine la position des trous sur une colonne du ruban et renvoie les sept bits correspondants.
    - Entrées : image (instance de la classe PIL.Image), n (entier, numéro de colonne sur le ruban)
    - Sortie : chaine (chaîne de caractères correspondant à l'écriture binaire d'un entier sur sept bits)
    """
    chaine = ""
    for k in range(8):
        if k != 3:
            if est_sombre(image.getpixel((315 + 120*n, 140 + 120*k))):
                chaine = "1" + chaine
            else:
                chaine = "0" + chaine
    return chaine

def ruban_vers_tableau(nom_de_fichier):
    """
    Transforme le dessin d'un ruban en un tableau d'entiers écrits sur sept bits.
    - Entrée : nom_de_fichier (chaîne de caractères, nom d'un fichier PNG)
    - Sortie : (tableau de chaînes de caractères, correspondant à des entiers écrits en binaire sur sept bits)
    """
    image = Image.open(nom_de_fichier)
    largeur, hauteur = image.size
    nb_caracteres = (largeur - 470) // 120  # longueur du message, 470 pixels de marges à gauche et à droite
    return [lecture_bits_colonne(image, n) for n in range(nb_caracteres)]

**Question 4 :** Expliquer le rôle des lignes `8` et `9` dans la définition de la fonction `lecture_bits_colonne`:

```python
for k in range(8):
    if k != 3:
```

**Réponse :** La boucle (ligne `8`) permet de parcourir (de haut en bas) les huit positions où se trouve potentiellement un trou. La condition (ligne `9`) permet de ne pas tenir compte du trou en quatrième position, c'est-à-dire du petit trou dont le rôle est de faire défiler le ruban.

**Question 5 :** Après avoir téléchargé le fichier `'ruban_long.png'` accessible [ici](https://ntoulzac.github.io/Cours-NSI-Terminale/premiere_NSI/Images/ruban_long.png), tester la fonction `ruban_vers_tableau` avec ce fichier PNG puis afficher le nombre de caractères présents sur le ruban.

In [3]:
liste = ruban_vers_tableau('ruban_long.png')
print(len(liste))

64


Le ruban peut donc désormais être transformé en un tableau contenant des chaînes de caractères, chaque chaîne étant composée de sept bits.

**Question 6 :** Définir une fonction `bits_vers_entier` qui prend en entrée une chaîne composée de sept bits et qui renvoie l'entier correspondant écrit en décimal.

Par exemple, l'appel `bits_vers_entier('1000010')` doit renvoyer l'entier `66`.

In [4]:
def bits_vers_entier(bits):
    """
    Donne l'écriture décimale d'un entier à partir de son écriture binaire sur sept bits.
    - Entrée : bits (chaîne de sept caractères, '0' ou '1')
    - Sortie : n (entier compris entre 0 et 127)
    """
    n = 0
    for k in range(7):
        if bits[k] == '1':
            n = n + 2**(6-k)
    return n

In [5]:
assert bits_vers_entier('1000010') == 66, "Problème..."

**Question 7 :** Définir une fonction `lecture_ruban` qui prend en entrée un nom de fichier PNG et qui renvoie sous forme de chaîne de caractères le message contenu dans le ruban.

In [6]:
def lecture_ruban(nom_de_fichier):
    """
    Extrait un message depuis un ruban perforé.
    - Entrée : nom_de_ficher (chaîne de caractères, nom d'un fichier PNG)
    - Sortie : chaine (chaîne de caractères, message extrait du ruban)
    """
    chaine = ""
    tab = ruban_vers_tableau(nom_de_fichier)
    for bits in tab:
        n = bits_vers_entier(bits)
        chaine = chaine + chr(n)
    return chaine

In [7]:
print(lecture_ruban("ruban_long.png"))

BrAvo ! VoUs aVez reUssI a deChifFrer le mEssaGe Sur lE rubAn...
